In [53]:
from selenium import webdriver
import requests
import io
from PIL import Image
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import csv

#The By module is typically used to specify the mechanism used to locate elements within a webpage, such as by ID, class name, CSS selector, etc. 

#And Keys module is used for sending special keyboard keys like Enter, Esc, etc. to web elements.

#The time module in Python provides various time-related functions. You can use it for tasks such as getting the current time, pausing or delaying the execution of code, measuring time durations, and more

In [47]:
path = "C:\\Users\\aditi\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"
wd=webdriver.ChromeService(executable_path=path) #creates an instance of Chrome WebDriver, which allows you to interact  with the Chrome browser.

driver = webdriver.Chrome(service=wd) # Create a new Chrome session and pass the service object
driver.maximize_window()
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")


In [ ]:

#image_url defines the url of the image we want to download
def download_image(download_path,image_url):
    try:
        image_content=requests.get(image_url).content 
    #gives the content of the image_url that is the image at that particular url
    
        image_file=io.BytesIO(image_content) 
    #converts the image_content to a ByteIO data type so that it gets saved in memory it stores binary data
    
        image=Image.open(image_file)
    #converts the binary data of image to a actual image
    
        file_path=os.path_join(download_path,'.jpg')
        with open(file_path,"wb") as f:# wb ---> write bytes
            image.save(f,"JPEG") 
            print('Success')    
    except Exception as e:
        print("Failed : ",e)
        


In [ ]:
def get_images(recipe_name,wd,delay,max_images):
    def scroll_down(wd):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(delay) #delay for scrolling
    driver.get("https://images.google.com/")
    search_box = driver.find_element(By.NAME,'q')
    search_box.send_keys(recipe_name)
    search_box.send_keys(Keys.RETURN)
    image_urls=set() #to avoid duplicate images
    while(len(image_urls)<max_images):
        scroll_down(wd)
        thumbnails = wd.find_elements(By.CLASS_NAME , "jlTjKd")
        for img in thumbnails[len(image_urls) : max_images]:
            try:
                img.click()
                time.sleep(delay)
            except:
                continue
            images=wd.find_elements(By.CLASS_NAME,"sFlh5c pT0Scc")
            for image in images:
                if(image.get_attribute("src") and 'http' in image.get_attribute)('src'):
                    image_urls.add(image.get_attribute('src'))
                    print(f"Found Image {len(image_urls)}")
    return image_urls          
                 

In [52]:
with open('recipes.csv','r') as f:
    reader=csv.DictReader(f)
    for row in reader:
        recipe_name = row['name']
        recipe_urls=get_images(recipe_name,driver,2,8)
        download_path=os.path_join('recipes_images/train',recipe_name)
        if not os.path.exists(download_path):
            os.makedirs(download_path)
        for url in recipe_urls:
            download_image(download_path,url):
        driver.quit()